### Hyperparameter Tuning Using Sweeps From WandB

**Sweeps: An Overview**

Running a hyperparameter sweep with W&B is very easy. There are just 3 simple steps: 

1. Define the sweep: we do this by creating a dictionary or a YAML file that specifies the parameters to search through, the search strategy, the optimization metric et all.

2. Initialize the sweep: with one line of code we initialize the sweep and pass in the dictionary of sweep configurations: sweep_id = wandb.sweep(sweep_config)

3. Run the sweep agent: also accomplished with one line of code, we call wandb.agent() and pass the sweep_id to run, along with a function that defines your model architecture and trains it: wandb.agent(sweep_id, function=train)



In [37]:
import wandb

In [38]:
wandb.login()

True

### Define Sweep

**Sweep strategy with sweep configuration**

There are multiple search methods.

1. grid
2. random
3. Bayesian

We'll go with Random

In [39]:
sweep_config = {
    'method': 'random'
    }

We are optimizing for loss metric.

In [40]:
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric


Hyperparameters to search through.

In [41]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'fc_layer_size': {
        'values': [128, 256, 512]
        },
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

Some parameter only for tracking, not for varying its value.

In [42]:
parameters_dict.update({
    'epochs': {
        'value': 1}
    })

For random search, all values of parameters are equally likely to be chosen on a given run.

Alternative, we can set a distribution for a parameters with mean and standard deviation.

In [43]:
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      }
    })

Sweep Configuration

In [44]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 32,
                               'q': 8},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'value': 1},
                'fc_layer_size': {'values': [128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


### Initialize the Sweep

In [45]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

Create sweep with ID: hv38vf9y
Sweep URL: https://wandb.ai/mlworks-org/pytorch-sweeps-demo/sweeps/hv38vf9y


### Defining ML Code

In [46]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config) as run:
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = run.config

        loader = build_dataset(config.batch_size)
        network = build_network(config.fc_layer_size, config.dropout)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            run.log({"loss": avg_loss, "epoch": epoch})


In [47]:
def build_dataset(batch_size):

    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307,), (0.3081,))])
    # download MNIST training dataset
    dataset = datasets.MNIST(".", train=True, download=True,
                             transform=transform)
    sub_dataset = torch.utils.data.Subset(
        dataset, indices=range(0, len(dataset), 5))
    loader = torch.utils.data.DataLoader(sub_dataset, batch_size=batch_size)

    return loader


def build_network(fc_layer_size, dropout):
    network = nn.Sequential(  # fully connected, single hidden layer
        nn.Flatten(),
        nn.Linear(784, fc_layer_size), nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(fc_layer_size, 10),
        nn.LogSoftmax(dim=1))

    return network.to(device)


def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0

    #with wandb.init() as run:
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        loss = F.nll_loss(network(data), target)
        cumu_loss += loss.item()

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

    return cumu_loss / len(loader)


Sweep agents are responsible for running an experiment with a set of hyperparameter values that you defined in your sweep configuration.

Sweep agent runs the train function five times.

In [48]:
wandb.agent(sweep_id, train, count=5)

wandb: Agent Starting Run: j1ra18n9 with config:
wandb: 	batch_size: 40
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.0033190048701530975
wandb: 	optimizer: adam


batch loss,█▅▄▄▃▂▅▅▆▅▄▃▃▅▅▁▃▄▁▇▂▂▂▂▂▂▁▂▅▅▁▂▂▄▂▃▂▁▂▂
epoch,▁
loss,▁
batch loss,0.34048
epoch,0
loss,0.43881


wandb: Agent Starting Run: sd6p9w2h with config:
wandb: 	batch_size: 80
wandb: 	dropout: 0.4
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.037798833522877495
wandb: 	optimizer: sgd


batch loss,█▅▄▄▄▃▂▃▃▃▃▃▂▃▂▂▂▂▁▂▃▂▂▃▂▂▃▂▂▁▁▂▁▁▂▂▁▂▂▂
epoch,▁
loss,▁
batch loss,0.36153
epoch,0
loss,0.52276


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 67s7v47x with config:
wandb: 	batch_size: 80
wandb: 	dropout: 0.4
wandb: 	epochs: 1
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.07731313136264874
wandb: 	optimizer: adam


batch loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁
loss,▁
batch loss,2.58977
epoch,0
loss,9.34864


wandb: Agent Starting Run: uop4s404 with config:
wandb: 	batch_size: 176
wandb: 	dropout: 0.4
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.05335684756381986
wandb: 	optimizer: adam


batch loss,▁██▇▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁
loss,▁
batch loss,0.92571
epoch,0
loss,3.04056


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 9uj50nmd with config:
wandb: 	batch_size: 144
wandb: 	dropout: 0.4
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.0015263282968578597
wandb: 	optimizer: sgd


batch loss,████▇█▇▇▇▇▇▆▆▆▆▆▆▅▆▆▅▅▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁
epoch,▁
loss,▁
batch loss,0.8384
epoch,0
loss,1.55919
